HD_Predict_failure
 * loads data from disk_bin.pkl file (created by HD_Extract_Features) which is a list of pandas data frames
 * tries some basic fits, uses extensively random forest: found that kneighbors and linearsvc were giving poor recall/precision, but more exploration needed

In [1]:
import os
import sys
import math
import struct
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import roc_curve
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn import model_selection
from sklearn import metrics
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import Range1d
from bokeh.charts import Bar, output_file, show
from bokeh.models import ColumnDataSource, Plot, Range1d
from bokeh.models import HoverTool
from bokeh.plotting import figure
import pickle
output_notebook()


Loading BokehJS ...

In [2]:
SELECTED_COLS = [u'smart_1_raw', u'smart_4_raw',u'smart_5_raw', u'smart_7_raw', 
        u'smart_9_raw', u'smart_10_raw',  u'smart_12_raw', u'smart_183_raw', 
        u'smart_184_raw', u'smart_187_raw', u'smart_188_raw', 
        u'smart_189_raw', u'smart_190_raw', u'smart_192_raw', u'smart_193_raw', 
        u'smart_194_raw', u'smart_197_raw', u'smart_198_raw', u'smart_199_raw', 
        u'smart_240_raw', u'smart_241_raw',u'smart_242_raw']

In [3]:
DIR_PATH = "data/ST4000DM000_data/"
DAY_BACK = 61 
#DAY_BACK = 5
NDAY = 0
x = np.load(DIR_PATH + 'train_d' + str(DAY_BACK -1) + '.npy')
y = np.load(DIR_PATH + 'label_d' + str(DAY_BACK - 1) + '.npy')
#z = np.load('data/lookback_d'+ str(DAY_BACK) + '.npy')
print np.shape(x)
print np.shape(y)

(35903, 61, 22)
(35903,)


In [ ]:
# np.shape(x)
# xp = [c[0:30] for c in x]
# print np.shape(xp)
# s = np.shape(x)
# x_full = x.reshape(s[0], s[1] * s[2])

In [5]:
def get_quality(estimator, x, y, day, pname="", do_save = False):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .34)
    y_pred = estimator.predict(x_test)
    TP = np.sum([1 if p==1 and t==1 else 0 for p,t in zip(y_pred,y_test)])
    FN = np.sum([1 if p==0 and t==1 else 0 for p,t in zip(y_pred,y_test)])
    FP = np.sum([1 if p==1 and t==0 else 0 for p,t in zip(y_pred,y_test)])
    TN = np.sum([1 if p==0 and t==0 else 0 for p,t in zip(y_pred,y_test)])
    # TP: number of failures correctly classified
    # FN: number of failures marked incorrectly as nominal
    # FP: number of nominal marked incorrectly as failure
    # TN: number of nominal correctly marked as nominal
    print("TP: ", TP, "   FN: ", FN, "   TN: ", TN, "   FP: ",  FP)
    print("Precision: ", TP/float(TP+FP), "Recall: ", TP/float(TP+FN))
    
    print classification_report(y_test, y_pred, target_names=['nominal','failure'])
    if do_save:
        y_prob = estimator.predict_proba(x_test)
        ypf = [i[1] for i in y_prob]
        #ypn = [i[0] for i in y_prob]
        precision_f, recall_f, thresholds_f = precision_recall_curve(y_test, ypf)
        #precision_n, recall_n, thresholds_n = precision_recall_curve(y_test, ypn)
        fpr, tpr, thresh = roc_curve(y_test, ypf)
        #fpr_n, tpr_n, thresh = roc_curve(y_test, ypn)
        np.save(DIR_PATH + "erf_precision_d" + str(day) + pname + ".npy", precision_f)
        np.save(DIR_PATH + "erf_recall_d" + str(day) + pname + ".npy", recall_f)
        np.save(DIR_PATH + "erf_fpr_d" + str(day) + pname + ".npy", fpr)
        np.save(DIR_PATH + "erf_tpr_d" + str(day) + pname + ".npy", tpr)
    
    

Found extra random forest works really damn well, and not too slow with this now small data set.
Hyper param search with CV.
Standard scaler brings each feature to ~zero mean and ~1 std var

In [ ]:
erf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('erf', ExtraTreesClassifier(class_weight="balanced_subsample",
                                 n_estimators=256, 
                                 min_samples_leaf=2,
                                 min_weight_fraction_leaf = 0,
                                 min_samples_split=12))
])    
    
param_grid = [
    {
        'erf__class_weight':  ["balanced", "balanced_subsample"],
        'erf__min_samples_split': np.arange(2, 16, 5),
        'erf__min_samples_leaf': np.arange(2, 16, 5),
        'erf__max_features': np.arange(19, np.shape(x[0])[1], 1),
        'erf__n_estimators': np.arange(32, 256, 64),
        'erf__min_impurity_split': [1e-6,1e-6,1e-5],
        'erf__min_weight_fraction_leaf':[0, .000005, .0000001]
    },
]

days = []#[0,3,2,1,0]
for day in days:
    print "Day: ", day, "\n"
    x_ = [item[day] for item in x]
    erf_best = GridSearchCV(erf_pipeline, cv=2, n_jobs=-1, param_grid=param_grid, scoring = 'f1', verbose=True)
    erf_best.fit(x_,y)
    print erf_best.best_params_
    print erf_best.best_score_
    erf_best = erf_pipeline.fit(x_,y)
    get_quality(erf_best, x_, y, day)

For each day of interest do a fit and save best params

In [ ]:
erf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('erf', ExtraTreesClassifier(class_weight="balanced_subsample",
                                 n_estimators=256, 
                                 min_samples_leaf=2,
                                 min_weight_fraction_leaf = 0,
                                 min_samples_split=12))
])    

all_days = range(np.shape(x)[1])[::-1]
days = [60, 50,  40, 30, 20, 10, 0]
for day in days:
    print "Day: ", day, "\n"
    x_ = [item[day] for item in x]
    erf_best = erf_pipeline.fit(x_,y)
    get_quality(erf_best, x_, y, day, do_save = True)

Now lets flatten the data so we can treat each day+feature as a unique feature, this gives us thousands of features! Lets look at what 60 to 0, 60 to 30 days would give us a prediction, 30 to 0,

In [ ]:
# np.shape(x)
# xp = np.asarray([c[0:30] for c in x])
# print np.shape(xp)
# s = np.shape(xp)
# xp_flat = xp.reshape(s[0], s[1] * s[2])
# print np.shape(xp_flat)

erf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('erf', ExtraTreesClassifier(class_weight="balanced_subsample",
                                 n_estimators=512, 
                                 min_samples_leaf=2,
                                 min_weight_fraction_leaf = 0,
                                 min_samples_split=12))
])    
    
param_grid = [
    {
        #'erf__class_weight':  ["balanced", "balanced_subsample"],
        'erf__min_samples_split': np.arange(12, 25, 5),
        #'erf__min_samples_leaf': np.arange(2, 16, 5),
        'erf__max_features': [512,600,660],
        #'erf__n_estimators': np.arange(32, 256, 64),
        'erf__min_impurity_split': [1e-6,1e-4],
        'erf__min_weight_fraction_leaf':[0, .00001]
    },
]

#the x array is backward rspt days: index 0 is day 60, index 60 is day 0
start_days = [0, 0, 30]
end_days = [30, 60, 0]
for n in range(len(start_days)):
    day = str(start_days[n]) + "_" + str(end_days[n])
    xp = np.asarray([c[start_days[n]:end_days[n]] for c in x])
    print np.shape(xp)
    s = np.shape(xp)
    xp_flat = xp.reshape(s[0], s[1] * s[2])
    print np.shape(xp_flat)
    erf_best = GridSearchCV(erf_pipeline, cv=2, n_jobs=-1, param_grid=param_grid, scoring = 'f1', verbose=True)
    erf_best.fit(xp_flat,y)
    print erf_best.best_params_
    print erf_best.best_score_
    #erf_best = erf_pipeline.fit(xp_flat,y)
    get_quality(erf_best, xp_flat, y, day, pname="back_flat",do_save=True)

(35903, 30, 22)
(35903, 660)
Fitting 2 folds for each of 36 candidates, totalling 72 fits


In [ ]:
# x_ = [item[4] for item in x]
# estimator = erf_best
# x_train, x_test, y_train, y_test = train_test_split(x_, y, test_size = .333)
# estimator.fit(x_train, y_train)

In [ ]:
# estimator = erf_best
# x_train, x_test, y_train, y_test = train_test_split(x_, y, test_size = .333)
# y_pred = estimator.predict(x_test)
# #for i in range(len(y_test)):
# #    if 
# TP = np.sum([1 if p==1 and t==1 else 0 for p,t in zip(y_pred,y_test)])
# FN = np.sum([1 if p==0 and t==1 else 0 for p,t in zip(y_pred,y_test)])
# FP = np.sum([1 if p==1 and t==0 else 0 for p,t in zip(y_pred,y_test)])
# TN = np.sum([1 if p==0 and t==0 else 0 for p,t in zip(y_pred,y_test)])
# # TP: number of failures correctly classified
# # FN: number of failures marked incorrectly as nominal
# # FP: number of nominal marked incorrectly as failure
# # TN: number of nominal correctly marked as nominal
# print("TP: ", TP, "   FN: ", FN, "   TN: ", TN, "   FP: ",  FP)
# print("Precision: ", TP/float(TP+FP), "Recall: ", TP/float(TP+FN))
# get_quality(estimator, x_, y, day)    

In [ ]:
# svc_pipeline = Pipeline([
#     ('svc', LinearSVC())#fit_intercept= False, dual=False))
# ])    
    
# param_grid = [
#     {
#         'svc__C': np.arange(2.0, 7.0, .5),
#         'svc__loss':  ["hinge", "squared_hinge"],
#         'svc__penalty': ["l1"],, "l2"], 
#         'svc__dual': [True, False],
#         'svc__max_iter': [1000,5000]
#     },
# ]

# svc_best = GridSearchCV(svc_pipeline, cv=3, n_jobs=-1, param_grid=param_grid, scoring = 'f1')
# svc_best.fit(x,y)
# mean_scores = np.array(svc_best.cv_results_['mean_test_score'])
# print mean_scores
# print grid_cv_result.best_estimator_
# rint grid_cv_result.best_index_

In [ ]:
# knc_pipeline = Pipeline([
#     ('knc', KNeighborsClassifier())
# ])    
    
# param_grid = [
#     {
#         'knc__n_neighbors': [2,4,8,16,32,64,128],
#         'knc__weights':  ["uniform", "distance"]
#         'metric': ["minkowski", "taxi", "jaccard"]
#         'knc__p': [1,2,3] # p is power for minkowksi metric
#     },
# ]
# knc_best = GridSearchCV(knc_pipeline, cv=3, n_jobs=-1, param_grid=param_grid, scoring = 'f1')
#knc_best.fit(x,y)
#precision, recall, thresholds = get_quality(knc_best, x,  y)

In [6]:
import matplotlib
cmap=plt.get_cmap("cubehelix")

title = "Precision-Recall"

plot = figure(title=title, width=600, height=600)
    
days = [60, 50,  40, 30, 20, 10, 0]
j = 0    
for day in days:
    c = float(j)/len(days)
    c = tuple([i*256 for i in cmap(c)[0:3]])
    print day, c
    precision = np.load(DIR_PATH + "erf_precision_d" + str(day) + ".npy")
    recall = np.load(DIR_PATH + "erf_recall_d" + str(day) + ".npy")
    
    source = ColumnDataSource(
        data=dict(
            precision=precision, 
            recall=recall,
            lookback_day = [day for i in recall]
            )
        )
        
        
    tmp = plot.line('recall', 'precision', line_width = 2.5, 
                    source = source, color =c)
    j += 1
        
plot.yaxis.axis_label = 'Precision'
plot.xaxis.axis_label = 'Recall'
plot.x_range = Range1d(-.005, 1.001)
plot.y_range = Range1d(0, 1.003)
plot.grid.grid_line_alpha = 0
plot.ygrid.grid_line_color = None
plot.toolbar.logo = None
plot.outline_line_width = 0
plot.outline_line_color = "white"
plot.yaxis.major_tick_line_width = 2
plot.xaxis.major_tick_line_width = 2
plot.xaxis.axis_line_width = 2
plot.yaxis.axis_line_width = 2
plot.title.text_font_size = '16pt'
plot.xaxis.axis_label_text_font_size = "14pt"
plot.xaxis.major_label_text_font_size = "14pt"
plot.yaxis.axis_label_text_font_size = "14pt"
plot.yaxis.major_label_text_font_size = "14pt"
show(plot)

60 (0.0, 0.0, 0.0)
50 (26.191424497494086, 35.719421099081998, 65.539080176875032)
40 (27.121564361226511, 97.529318696840477, 69.158684884221557)
30 (105.11693498042271, 122.99463818668097, 48.41354951112919)
20 (200.42709459278251, 123.28525686397651, 124.60147703900824)
10 (205.981909229476, 162.96277937412725, 225.1924071149088)
0 (199.05557438408854, 226.31404854303867, 241.97140460403332)


### How complete are the results? 
In information retrieval, precision is a measure of result relevancy, while recall is a measure of how many truly relevant results are returned. The commonly used reciever operator characteristic curve measures the fraction of true positives out of the true positives results verusus the fraction of false positives out of the true negatives. However in this situation true negative results are uninteresting, these are nominal operating hard drives. In this situation failures are rare (confusingly a failure here is a "positive prediction") and so a good a priori assumption for all hard drives is that it will not fail, indeed this would make you more than about 97% accurate for this data set. A "good" model might (rightly so) make very few postive predictions and make no false postive predictions, thus obtaining an incredible area under the curve in a receiver operator characteristic measure.

The precision recall curve is of more interest perhaps. High precision means we are returing accurate results(TP/(TP + FP)). High recall means we are identifiyng the majority of postive results (TP / (TP + FN)). High recall but low precision returns many positive predictions, but most of those predictions are incorrect. And conversely high precision, but low recall returns few postive predictions, but most of the resutls returned are correct.

### How useful are the results?
   
In the precision recall curve we can consider the effect of lowering the threshold of a classifier. A high threshold means results may all be true positives, which will actually increase precision. On the other hand if the threshold was too low (or approriate), then lowering the threshold will introduce false positives and decrease precision.

Storing data on hard drives is fundamentally risky. Even if we could predict the exact day of hard drive failure with machine learning it would not be enough. An asteroid or a giant praying mantis could destroy the data center and all the hard drives. With wise respect for the unexpected, we see that it would be best to simply store everything in multiple locations.  This is exactly what backup data services do, and this is exactly what you should do too. Store important data in multiple physical locations!

That being said, can we use these predictions to increase the efficeny of storage or decrease costs of storage? Generally the answer here is no, just like our null hypothesis. For example, if you could predict when your car was going to fail, but had no economic cost effective way of preventing that failure, how much would that prediction be worth? Maybe it is worth the price of your roadtrip vacation, but not the price of the car. Predicting a hard drive failure before it happens could stop brief downtimes of servers and reduce user service interruption, however most of these systems are already in redundant failback mode for many deployments. If we can predict hard drives that are about to fail we can remove those hard drives just before they fail, but we still have to pay for monitoring, replacement labor, and new hard drives. In addition, if we make false postive predictions we are removing hard drives which are perfectly good and that would be a bad cost to incur. Since we are going to have that data backed up somewhere else anyways it may be best to let the drive run till failure then replace it and pull back the data from the other backups. The ultimate choice whether to remove drives that are in emminent failure is a logistic and economic decision.